In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib
import random
import os
import deepdish as dd
from collections import defaultdict
#SCIPY
from scipy.optimize import fsolve
import scipy.optimize as opt

Using matplotlib backend: MacOSX


In [2148]:
import seaborn; seaborn.set()

In [12]:
def Zipf_Mandelbrot_CDF(n, alpha, beta=2.7):
    """Computes Zipf-Mandelbrot cumulative distribution function"""
    x = np.arange(1, n + 1) + beta
    mandelbrot_law = np.power(x, -alpha)
    zeta = np.cumsum(mandelbrot_law )
    return zeta / zeta[-1]

def Zipf_CDF(n, alpha):
    """Computes Zipf cumulative distribution function.
    Each percentage corresponds to word index in array"""
    zipf_law = np.power(np.arange(1, n + 1), -alpha)
    zeta = np.cumsum(zipf_law)
    return zeta / zeta[-1]

def Zipf_CDF_compressed(n, alpha, n_red=1000):
    """Computes Zipf cumulative distribution function.
    It compresses real long interval n into a smaller n_red
    by conserving relative percentages by intervals
    Each percentage corresponds to word index in array"""

    zipf_law = np.power(np.arange(1, n + 1), -alpha)

    zeta = np.cumsum(zipf_law)
    zeta = zeta / zeta[-1]
    interv_div = np.linspace(1, n, n_red + 1).astype(np.int64)

    # zeta = np.array([zeta[i1-1] for i1 in interv_div[1:]])
    zeta = zeta[interv_div[1:] - 1]
    return zeta

def Zipf_Mand_CDF_compressed(n, alpha, beta=2.7, n_red=1000):
    """Computes Zipf cumulative distribution function.
    It compresses real long interval n into a smaller n_red
    by conserving relative percentages by intervals
    Each percentage corresponds to word index in array"""
    
    x = np.arange(1, n + 1) + beta
    zipf_mand_law = np.power(x, -alpha)

    zeta = np.cumsum(zipf_mand_law)
    zeta = zeta / zeta[-1]
    interv_div = np.linspace(1, n, n_red + 1).astype(np.int64)

    # zeta = np.array([zeta[i1-1] for i1 in interv_div[1:]])
    zeta = zeta[interv_div[1:] - 1]
    return zeta



def Zipf_Mand_3S_CDF(n, alpha_1=1.17, alpha_2=1.67, alpha_3=2.33,
                           beta=7.1, c1=2153013.1,c2=10052.1,c3=167.25, c4 =98.83,
                           N1= 100, N2 = 2000):
    """Computes Zipf-Mandelbrot cumulative distribution function in three stages"""
    
    x = np.arange(1, n + 1) + beta
    v1 = c1 * np.power(x[:N1], -alpha_1)
    v2 = N1 ** alpha_2 * c2 * np.power(x[N1:N2], -alpha_2) + c4
    v3 = N2 ** alpha_3 * c3 * np.power(x[N2:], -alpha_3)

    mandelbrot_law_3S = np.concatenate((v1, v2, v3))
    zeta = np.cumsum(mandelbrot_law_3S)
    return zeta / zeta[-1]

# IDEA : to model vocab_size vs age dependency, play both with n and n_red in following function
# Use factor for n, n_red ???
def Zipf_Mand_3S_CDF_comp(n, alpha_1=1.17, alpha_2=1.67, alpha_3=2.33,
                          beta=7.1, c1=2153013.1, c2=10052.1, c3=167.25, c4 =98.83,
                          N1= 100, N2 = 2000, n_red=1000):
    """Computes Zipf-Mandelbrot cumulative distribution function in three stages
       It compresses real long interval n into a smaller n_red
       by conserving relative percentages by intervals"""
    
    x = np.arange(1, n + 1) + beta
    v1 = c1 * np.power(x[:N1], -alpha_1)
    v2 = N1 ** alpha_2 * c2 * np.power(x[N1:N2], -alpha_2)
    v3 = N2 ** alpha_3 * c3 * np.power(x[N2:], -alpha_3)

    mandelbrot_law_3S = np.concatenate((v1, v2, v3))
    zeta = np.cumsum(mandelbrot_law_3S)
    zeta = zeta / zeta[-1]
    
    interv_div = np.linspace(1, n, n_red + 1).astype(np.int64)
    zeta = zeta[interv_div[1:] - 1]
    return zeta


def randZipf(zipf_cum_distr, numSamples):
    """fast computation of array of Zipf samples with dim = numSamples
    It needs Zipf CDF as input"""
    unif_random_array = np.random.random(numSamples)
    return np.searchsorted(zipf_cum_distr, unif_random_array)

### BNC corpus research

In [355]:
import re
import deepdish as dd
from xml.etree.ElementTree import ElementTree
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
%matplotlib

Using matplotlib backend: MacOSX


In [26]:
#import stemmers and progress bar
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
porter = PorterStemmer()
porter_sb = SnowballStemmer('english')


import pyprind

In [56]:
os.chdir("/Users/PG/Paolo/python_repos/language_proj/BNC/2554/download/Texts")

In [7]:
# Reduced DB
os.chdir("/Users/PG/Paolo/python_repos/language_proj/BNC/2553/2553/download/Texts/news")
# news 
xml_news_elems = []
for file in os.listdir():
    xml_news_elems.append(ElementTree().parse(file))

all_news_words = []
for xml_elem in xml_news_elems:
    all_news_words.extend([s.text for s in xml_elem.findall('wtext/div/p/s/w')])

In [8]:
#Reduced DB
os.chdir("/Users/PG/Paolo/python_repos/language_proj/BNC/2553/2553/download/Texts/dem")
# conversation
xml_elems = []
for file in os.listdir():
    xml_elems.append(ElementTree().parse(file))

all_words = []
for xml_elem in xml_elems:
    all_words.extend([s.text for s in xml_elem.findall('stext/div/u/s/w')])

In [9]:
x = [100,1000,10000,50000,100000,250000, 500000, 750000, 999000]
#y1 = [np.unique(all_words[:lens]).shape[0]/len(all_words[:lens]) for lens in x]
y2 = [np.unique(all_words[:lens]).shape[0] for lens in x]
y3 = [np.unique(all_news_words[:lens]).shape[0] for lens in x]

plt.plot(x,y2, label="conv")
plt.plot(x,y3, label ="news")
plt.legend()
plt.grid()

In [55]:
# switch to full DB directory
os.chdir("/Users/PG/Paolo/python_repos/language_proj/BNC/2554/download/Texts")

In [28]:
# get list of file names
path_filename_list=[]
for dirpath, dirnames, filenames in os.walk("."):
    for filename in [f for f in filenames if f.endswith(".xml")]:
        path_filename_list.append(os.path.join(dirpath, filename))

In [227]:
l_file_paths = path_filename_list[-10:]

In [258]:
xml_obj = ElementTree().parse(l_file_paths[8])

In [7]:
# conversation full CORPUS
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

import pyprind
pbar = pyprind.ProgBar(749)

all_full_conv_words = []
for file in path_filename_list[3300:4049]:
    xml_obj = ElementTree().parse(file)
    try:
        all_full_conv_words.extend([s.text.replace(" ","") 
                                    for s in xml_obj.findall('stext/div/u/s/w')
                                    if s.text
                                    and not re.findall(r"[^a-zA-Z'-]+", s.text.replace(" ",""))])
        pbar.update()
    except:
        pass
#stemmed_s_words = [porter.stem(word) for word in all_full_conv_words]

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:22


In [1873]:
# written texts full corpus
import pyprind
pbar = pyprind.ProgBar(700)
full_written_words = []
for file in path_filename_list[:700]:
    xml_obj = ElementTree().parse(file)
    full_written_words.extend([w.text.replace(" ","") 
                               for w in xml_obj.findall('wtext/div/p/s/w')
                               if not re.findall(r"[^a-zA-Z']+", w.text)])
    pbar.update()

stemmed_w_words = [porter.stem(word) for word in full_written_words]  

In [42]:
# CODE TO GET WORDS FROM CONVERSATIONAL AND WRITTEN ENGLISH FROM FULL DATABASE
pbar = pyprind.ProgBar(len(path_filename_list))

all_full_s_words = []
all_full_w_words = []
for file in path_filename_list:
    xml_obj = ElementTree().parse(file)
    if xml_obj.findall('stext/div/u/s/w'):
        all_full_s_words.extend([s.text.replace(" ","") 
                                 for s in xml_obj.findall('stext/div/u/s/w') 
                                 if s.text
                                 and not re.findall(r"[^a-zA-Z']+", s.text.replace(" ",""))])
        pbar.update()
    else:
        all_full_w_words.extend([w.text.replace(" ","") 
                                 for w in xml_obj.findall('wtext/div/p/s/w')
                                 if w.text
                                 and not re.findall(r"[^a-zA-Z']+", w.text.replace(" ",""))])
        pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 01:03:54


###### SAVE RELEVANT INFO

In [45]:
# SAVE AND LOAD FULL DB WORDS
# words_full_BNC = {'w':np.array(all_full_w_words), 
#                   's':np.array(all_full_s_words)}
# dd.io.save('words_full_BNC.h5', words_full_BNC)

words_full_BNC = dd.io.load('words_full_BNC.h5')

In [55]:
# STEMMING AND UNIQUE COUNTS SPOKEN
stemmed_s_words = [porter.stem(word).lower() for word in words_full_BNC['s']]

stemmed_sb_s_words = [porter_sb.stem(word).lower() for word in words_full_BNC['s']]

uq_s_words = np.unique(stemmed_s_words, return_counts=True)

uq_sb_s_words = np.unique(stemmed_sb_s_words, return_counts=True)

In [57]:
# STEMMING AND UNIQUE COUNTS WRITTEN
stemmed_w_words = [porter.stem(word).lower() for word in words_full_BNC['w'][:5000000]]

stemmed_sb_w_words = [porter_sb.stem(word).lower() for word in words_full_BNC['w'][:5000000]]

uq_w_words = np.unique(stemmed_w_words, return_counts=True)

uq_sb_w_words = np.unique(stemmed_sb_w_words, return_counts=True)

In [59]:
#SAVE STEMMED WORDS
stemmed_words_full_BNC = {'w':np.array(stemmed_w_words), 
                          's':np.array(stemmed_s_words)}
dd.io.save('stemmed_words_full_BNC.h5', stemmed_words_full_BNC)

stemmed_words_SB_full_BNC = {'w':np.array(stemmed_sb_w_words), 
                             's':np.array(stemmed_sb_s_words)}
dd.io.save('stemmed_words_SB_full_BNC.h5', stemmed_words_SB_full_BNC)

In [60]:
#SAVE UNIQUE WORDS
unique_words_full_BNC = {'w':uq_w_words, 
                         's':uq_s_words}
dd.io.save('unique_words_full_BNC.h5', unique_words_full_BNC)

unique_words_SB_full_BNC = {'w':uq_sb_w_words, 
                            's':uq_sb_s_words}
dd.io.save('unique_words_SB_full_BNC.h5', unique_words_SB_full_BNC)

##### LOAD DATA and COMPUTE FIT COEFFICIENTS of PROBAB DISTRIBUTION

In [5]:
# switch to full DB directory
os.chdir("/Users/PG/Paolo/python_repos/language_proj/BNC/2554/download/Texts")

In [7]:
# LOAD UNIQUE and STEMMED WORDS
#uq_words = dd.io.load('unique_words_full_BNC.h5')
uq_SB_words = dd.io.load('unique_words_SB_full_BNC.h5')
stemmed_SB_words = dd.io.load('stemmed_words_SB_full_BNC.h5')

In [1789]:
x = [100,1000,10000,50000,100000,250000, 500000, 750000, 
     1000000, 2000000, 3000000, 4000000, 4800000]
y_s = [np.unique(stemmed_SB_words['s'][:lens]).shape[0] for lens in x]
y_w = [np.unique(stemmed_SB_words['w'][:lens]).shape[0] for lens in x]

In [1790]:
plt.plot(x, y_s, label= 'spoken')
plt.plot(x, y_w, label = 'written')
plt.legend()
plt.grid('on')

Check 100 most relevant words in 's', 'w' vocabs

In [1793]:
uq_SB_words['s'][0][np.argsort(uq_SB_words['s'][1])[::-1]][:100], uq_SB_words['w'][0][np.argsort(uq_SB_words['w'][1])[::-1]][:100]

In [ ]:
# ZIPF - HEAPS CURVE FITTING -> GET PARAMETERS VALUES
def heaps_fun(x, k, beta):
    x = np.array(x)
    return k * x ** beta

x_h = np.array([1,10,100,1000,10000,100000,1000000,4000000])
y_h = [np.unique(stemmed_SB_words['s'][:lens]).shape[0] for lens in x_h]

(k_h, beta_h),_ = opt.curve_fit(heaps_fun, x_h, y_h, p0=(50, 0.5))

xx=np.linspace(1,4000000, 1000)
plt.plot(xx, k_h * xx ** beta_h)
plt.grid()

FUNCTIONS TO FIT POWER LAWS TO DATA

In [13]:
def zipf_fit(x, alpha, c):
    #x = x + beta
    #return - alpha * np.log(x)
    return c * np.power(x, -alpha)

def zipf_mand_fit(x, alpha, beta, c):
    #x = x + beta
    #return - alpha * np.log(x)
    x = x + beta
    return c * np.power(x, -alpha)

def zipf_mand_fit_2S(r, alpha1, alpha2, beta, c1, c2):
    N=2000

    r = r + beta
    v1 = c1 * np.power(r[:N], -alpha1)
    v2 = N**alpha2 * c2 * np.power(r[N:], -alpha2)
    eps = np.abs(c1 * np.power(r[N-1], -alpha1) - N**alpha2 * c2 * np.power(r[N], -alpha2))
    
    f_union = np.concatenate((v1, v2))
    #f_union[N-1:N+1] += 10*eps
        
    return f_union + eps

def zipf_mand_fit_3S(r, alpha1, alpha2, alpha3, beta, c1, c2, c3, c4):
    
    N1 = 100
    N2 = 2000

    r = r + beta
    v1 = c1 * np.power(r[:N1], -alpha1)
    v2 = N1**alpha2 * c2 * np.power(r[N1:N2], -alpha2) + c4 # need extra DOF to fit conditions
    v3 = N2**alpha3 * c3 * np.power(r[N2:], -alpha3)
    
#     eps1 = np.abs(c1 * np.power(r[N1-1], -alpha1) - y_s[100])
#     eps2 = np.abs(N1**alpha2 * c2 * np.power(r[N1], -alpha2) + c4 - y_s[100])
#     eps3 = np.abs(N1**alpha2 * c2 * np.power(r[N2-1], -alpha2) + c4 - y_s[2000])
#     eps4 = np.abs(N2**alpha3 * c3 * np.power(r[N2], -alpha3) - y_s[2000])
    
#     eps1 = np.abs(c1 * np.power(r[N1-1], -alpha1) 
#                   - N1**alpha2 * c2 * np.power(r[N1], -alpha2) - c4)
    
#     eps2 = np.abs(N1**alpha2 * c2 * np.power(r[N2-1], -alpha2) + c4 
#                   - N2**alpha3 * c3 * np.power(r[N2], -alpha3))
    
    f_union = np.concatenate((v1, v2, v3))
    
#     f_union[N1-1] += eps1
#     f_union[N1] += eps2
    
#     f_union[N2-1] += eps3
#     f_union[N2] += eps4
    
    #f_union[N1-1: N1 + 1] += 10 * eps1
    #f_union[N2-1:N2+1] += 10 * eps2
    
    return f_union #+ eps1 + eps2 #+ eps3 + eps4

In [1911]:
k2,k4=10000,100
plt.plot(x_s,100*1.6*k2 * np.power(x_s, -1.6) + k4)
plt.xscale('log')
plt.yscale('log')

Define DATA to be FIT ( important to do it properly...)

In [8]:
n_s = 24000
n_w = 52000
x_s = np.arange(1, n_s + 1)
x_w = np.arange(1, n_w + 1)

y_s = np.sort(uq_SB_words['s'][1])[::-1][:n_s]
y_w = np.sort(uq_SB_words['w'][1])[::-1][:n_w]

In [1759]:
#ONE REGIME FIT (SPOKEN)
(alpha_s,beta_s, c_s), _ = opt.curve_fit(zipf_mand_fit, x_s, y_s, p0=(1.3, 3, 1000))
(alpha_s,beta_s, c_s)

(1.2979192327416069, 8.7774134919893907, 3511560.4378326912)

In [1760]:
# TWO REGIMES FIT (SPOKEN)
(alpha_s1, alpha_s2, beta_s_imp, c_s1, c_s2), _ = opt.curve_fit(zipf_mand_fit_2S, 
                                                            x_s, y_s, 
                                                            p0=(1.3,2.5, 3, 100000, 100))
(alpha_s1, alpha_s2, beta_s_imp, c_s1, c_s2)

(1.2384066155491573,
 1.8735935624145901,
 7.8422120761183924,
 2727267.9722089712,
 223.45785615292965)

In [2258]:
# THREE REGIMES FIT (SPOKEN)
sigma =np.ones(len(y_s))
sigma[[99, 100, 101, 500, 1700, 1999, 2000, 2001, 2002,  -1]] = 0.001
(a3S_1, a3S_2,a3S_3,beta_3S, c3S_1, c3S_2, c3S_3, C4_S), _ = opt.curve_fit(zipf_mand_fit_3S, 
                                                                           x_s, y_s, 
                                                                           p0=(1., 1.3, 2.5, 
                                                                               3, 
                                                                               100000, 1000, 100, 10),
                                                                           sigma=sigma)
(a3S_1, a3S_2,a3S_3,beta_3S, c3S_1, c3S_2, c3S_3, C4_S)


# (a3S_1, a3S_2,a3S_3,beta_3S, c3S_1, c3S_2, c3S_3), _ = opt.curve_fit(zipf_mand_fit_3S, 
#                                                                            x_s, y_s, 
#                                                                            p0=(1., 1.3, 2.5, 
#                                                                                3, 
#                                                                                100000, 1000, 100),
#                                                                            sigma=sigma)
# (a3S_1, a3S_2,a3S_3,beta_3S, c3S_1, c3S_2, c3S_3)

(1.2222786840696638,
 1.4954601266269689,
 1.8986458118523217,
 7.7972793355069152,
 2613014.9799408969,
 9668.5668750285349,
 122.1289147117548,
 11.429085154568304)

In [2318]:
# DATA vs FIT COMPARISON (SPOKEN) 
plt.plot(x_s, y_s, label = 'data')
plt.plot(zipf_mand_fit_3S(x_s, a3S_1, a3S_2, a3S_3, beta_3S, c3S_1, c3S_2, c3S_3, C4_S), 
         label = '3S_fit')

# plt.plot(zipf_mand_fit_2S(x_s, alpha_s1, alpha_s2, beta_s_imp, c_s1, c_s2), label='2S_fit')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('word rank')
plt.ylabel('word frequency')
plt.title('Data vs fit in spoken words corpus')
plt.grid('on')
plt.grid(True,which="both",ls="-")
plt.legend()

CONCLUSION : Z-M fitting overestimates frequency from r = 1000 onwards
There are two regimes in linguistic power law. 
Shift occurs around r = 2000-3000
See papers by MONTEMURRO : 'Beyond the Zipf-Mandelbrot law in quantitative linguistics'
and FERRER-I-CANCHO and SOLE : 'Two regimes in frequency of words'
Real fitting is cumbersome -> Need to simplify

Visualize power law dependency on vocab size

In [1802]:
for n_red in [50,1000,5000,15000,24000]:
    interv_div = np.linspace(1, 24000, n_red + 1).astype(np.int64)
    new_ys = y_s[interv_div[1:] - 1]
    plt.plot(range(n_red),new_ys, label = str(n_red) + 'words')
    
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [1719]:
# 2S WRITTEN FIT
(alpha_w1, alpha_w2, beta_w_imp, c_w1, c_w2), _ = opt.curve_fit(zipf_mand_fit_2S, 
                                                            x_w, y_w, 
                                                            p0=(1.,2., 3, 100000, 100))
(alpha_w1, alpha_w2, beta_w_imp, c_w1, c_w2)

(0.9091269738214558,
 1.4817794053535509,
 0.33552797658385036,
 413393.53629757592,
 412.73527425955137)

In [2139]:
# 3S WRITTEN FIT
sigma =np.ones(len(y_w))
sigma[[99, 100, 101, 500, 1700, 1999, 2000, 2001, 2002, -1]] = 0.001
(a3S_w1, a3S_w2,a3S_w3,beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w), _ = opt.curve_fit(zipf_mand_fit_3S, 
                                                            x_w, y_w, 
                                                            p0=(1.,1.3,2.5, 3, 100000, 1000,100, 10),
                                                            sigma=sigma)
(a3S_w1, a3S_w2,a3S_w3,beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w)

(0.98779924555558052,
 0.78013700474286507,
 1.6496670400187725,
 0.60344865923415125,
 501780.35693374212,
 5544.6062437143801,
 279.60203407490104,
 -259.15257421842836)

In [2140]:
# DATA vs FIT COMPARISON WRITTEN
plt.plot(x_w,y_w, label = 'data')

plt.plot(zipf_mand_fit_3S(x_w, a3S_w1, a3S_w2,a3S_w3,beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w), 
         label = '3_region_fit')
plt.title('Data vs fit in written words corpus')
plt.xscale('log')
plt.yscale('log')
plt.legend()

###### COMPARE SPOKEN vs WRITTEN POWER LAWS

In [2321]:
plt.plot(zipf_mand_fit_3S(x_s, a3S_1, a3S_2, a3S_3, beta_3S, c3S_1, c3S_2, c3S_3, C4_S), 
         label = '3R_fit_S')
plt.plot(zipf_mand_fit_3S(x_w, a3S_w1, a3S_w2,a3S_w3,beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w), 
         label = '3R_fit_W')

# plt.plot(zipf_mand_fit_2S(x_s, alpha_s1, alpha_s2, beta_s_imp, c_s1, c_s2), label='2S_fit')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('word rank')
plt.ylabel('word frequency')
plt.title('Data vs fit in spoken words corpus')
plt.grid('on')
plt.grid(True,which="both",ls="-")
plt.legend()

### Exploring correct order of COMPRESSED merged s, w VOCAB array

In [342]:
# SET DIFFERENCE WRITTEN-SPOKEN
set_diff=np.setdiff1d(uq_SB_words['w'][0], uq_SB_words['s'][0], assume_unique=True)

In [343]:
# INTERSECTION WRITTEN-SPOKEN
set_inters=np.intersect1d(uq_SB_words['w'][0], uq_SB_words['s'][0], assume_unique=True)

In [641]:
w_s = np.array(['aa','xx','cd','cc','ww','yy'])
w_s_counts = np.array([33,22,7,3,2,1], dtype=np.uint32)

w_s_dict = dict(zip(w_s, w_s_counts))

w_w = np.array(['cc','bb','aa','cd','tw','ww'])
w_w_counts = np.array([25,18,14,9,5,1], dtype=np.uint32)

w_w_dict = dict(zip(w_w, w_w_counts))

In [656]:
# union
union_vocab = np.union1d(w_s, w_w)
union_vocab_c = np.zeros(union_vocab.shape[0], dtype=np.uint32)



# get total counts of intersect

# union_vocab_c[np.in1d(union_vocab, w_s)] += w_s_counts
# union_vocab_c[np.in1d(union_vocab, w_w)] += w_w_counts

for idx, word in enumerate(union_vocab):
    try:
        c = w_s_dict[word]
        union_vocab_c[idx] += c
    except:
        pass
    
for idx, word in enumerate(union_vocab):
    try:
        c = w_w_dict[word]
        union_vocab_c[idx] += c
    except:
        pass
    
print(union_vocab, union_vocab_c)

['aa' 'bb' 'cc' 'cd' 'tw' 'ww' 'xx' 'yy'] [47 18 28 16  5  3 22  1]


In [657]:
union_vocab=union_vocab[np.argsort(union_vocab_c)[::-1]]
union_vocab_c = union_vocab_c[np.argsort(union_vocab_c)[::-1]]
union_vocab, union_vocab_c

(array(['aa', 'cc', 'xx', 'bb', 'cd', 'tw', 'ww', 'yy'], 
       dtype='<U2'), array([47, 28, 22, 18, 16,  5,  3,  1], dtype=uint32))

In [607]:
# get total counts of intersect
np.in1d(union_vocab, w_w), np.in1d(w_w, union_vocab)

(array([ True,  True,  True,  True,  True,  True, False, False], dtype=bool),
 array([ True,  True,  True,  True,  True,  True], dtype=bool))

In [550]:
s_in_w = np.in1d(w_s,w_w)

In [551]:
w_in_s = np.in1d(w_w,w_s)

In [553]:
w_s[s_in_w]

array(['aa', 'bb', 'cd', 'ww'], 
      dtype='<U2')

In [559]:
w_w_counts[w_in_s] += w_s_counts[s_in_w]

In [560]:
w_w_counts

array([58, 40, 14, 16,  5,  3])

In [136]:
tot_common_counts = w_w_counts[w_in_s] + w_s_counts[s_in_w]
tot_common_counts

array([58, 40, 16,  3])

In [124]:
# add common elements counts from 's' to 'w'  
w_w_counts[w_in_s] += w_s_counts[s_in_w]

In [128]:
w_s_counts[~np.in1d(w_s,w_w)]

array([3, 1])

In [129]:
w_w_counts[~np.in1d(w_w,w_s)]

array([14,  5])

In [117]:
w_w_counts

array([58, 40, 14, 16,  5,  3])

In [118]:
w_w[w_in_s]

array(['aa', 'bb', 'cd', 'ww'], 
      dtype='<U2')

In [108]:
w_s_counts

array([33, 22,  7,  3,  2])

In [100]:
v1 = np.array([2,4,6,8])
v2 = np.array([10,20])

In [101]:
v1[[1,3]] += v2

In [104]:
s_in_w

array([ True,  True,  True, False,  True], dtype=bool)

In [138]:
np.searchsorted(tot_common_counts[::-1], w_s_counts[~np.in1d(w_s,w_w)], side = "right")

array([1, 0])

In [139]:
tot_common_counts[::-1]

array([ 3, 16, 40, 58])

In [140]:
w_s_counts[~np.in1d(w_s,w_w)]

array([3, 1])

In [143]:
np.concatenate((tot_common_counts, w_s_counts[~np.in1d(w_s,w_w)]))

array([58, 40, 16,  3,  3,  1])

In [144]:
np.concatenate((tot_common_counts, w_w_counts[~np.in1d(w_w,w_s)]))

array([58, 40, 16,  3, 14,  5])

In [145]:
xx=np.concatenate((tot_common_counts, w_w_counts[~np.in1d(w_w,w_s)]))
np.argsort(xx)

array([3, 5, 4, 2, 1, 0])

In [188]:
np.sort(uq_SB_words['s'][0])[:20000]

array(["'", "'a", "'d", ..., 'solhal', 'solicit', 'solicitor'], 
      dtype='<U104')

In [189]:
np.sort(uq_SB_words['w'][0])[:40000]

array(["'", "''", "'a", ..., 'rowley', 'rowlock', 'rowntre'], 
      dtype='<U25')

In [232]:
sort_s = uq_SB_words['s'][0][np.argsort(uq_SB_words['s'][1])[::-1]][:20000]
sort_s_c=uq_SB_words['s'][1][np.argsort(uq_SB_words['s'][1])[::-1]][:20000]

sort_w = uq_SB_words['w'][0][np.argsort(uq_SB_words['w'][1])[::-1]][:40000]
sort_w_c=uq_SB_words['w'][1][np.argsort(uq_SB_words['w'][1])[::-1]][:40000]

In [170]:
np.intersect1d(sort_s,sort_w).shape

(15692,)

In [195]:
np.in1d(sort_s,sort_w).shape # s-based

(20000,)

In [196]:
np.in1d(sort_w,sort_s).shape # w-based

(40000,)

In [187]:
plt.plot(np.sort(uq_SB_words['s'][1])[::-1][:20000][np.in1d(sort_s,sort_w)[:20000]])
plt.xscale('log')
plt.yscale('log')

In [204]:
dd=np.nonzero(np.in1d(sort_s,sort_w))[0]

In [206]:
plt.plot(dd)

In [231]:
sort_s[~np.in1d(sort_s,sort_w)][:100], sort_s_c[~np.in1d(sort_s,sort_w)][:100]

(array(['mhm', 'int', 'urgh', 'tt', 'aargh', 'bryoni', 'bom', 'marg', 'umm',
        'yum', 'kath', 'hiya', 'nope', 'ding', "an'al", 'gemma', 'lego',
        'poo', 'sshh', 'oy', 'ahhh', 'jehovah', 'pauli', 'cath', 'shel',
        'uhum', 'gwendolin', 'hayley', 'heidi', 'snog', 'foxi', 'shrimpi',
        'awl', 'fahrenheit', 'hallo', 'ey', 'deana', 'dinda', 'lari',
        'pott', 'ehm', 'golli', "t'", 'choo', 'ner', 'joell', 'winsi',
        'corrinn', 'plonker', 'whatsernam', 'whoo', 'southwold', 'tarrah',
        'frig', 'woh', 'markham', 'bedg', 'krispi', 'lyndsey', 'ashington',
        'magilton', 'kaley', 'almondsburi', 'chuff', 'scanner', 'melissa',
        'dong', 'gorbachov', 'netto', 'quoit', 'urc', 'stowmarket',
        'westbound', 'poxi', 'timmi', 'popsey', 'thunderbird', 'macro',
        'yuck', 'buby', 'lawford', 'ohhh', 'twat', 'nanna', 'shandi',
        'whee', 'arf', 'skint', 'mikila', 'dempsey', 'hatti', 'kimmi',
        'racetrack', 'carterton', 'beezer', 'miocenia'

In [234]:
sort_w[~np.in1d(sort_w,sort_s)][:100], sort_w_c[~np.in1d(sort_w,sort_s)][:100]

(array(['mungo', 'ec', 'nordern', 'erika', 'gazzer', 'engel', 'tammuz',
        'mcleish', 'wexford', 'moran', 'br', 'herr', 'noriega', 'panama',
        'frau', 'karajan', 'nhs', 'ii', 'alida', 'pragu', 'roirbak',
        'leila', 'hatton', 'ferranti', 'dorothea', 'elisabeth',
        'nonconformist', 'menzi', 'hir', 'clarissa', 'gaili', 'tremayn',
        'hobb', 'caspar', 'warsaw', 'ira', 'omi', 'holliday', 'itv', 'bodo',
        'marxist', 'jos', 'depict', 'libel', 'lb', 'harker', 'guberniia',
        'cullam', 'yeo', 'ceausescu', 'masha', 'qc', 'gm', 'fanshaw',
        'clow', 'natwest', 'eurotunnel', 'tolkien', 'dti', 'panamanian',
        'surkov', 'rozanov', 'jahsaxa', 'knighton', 'koon', 'crevecoeur',
        'havel', 'anc', 'mitterrand', 'civilis', 'pr', 'faldo', 'cooney',
        'malamut', 'kinship', 'plo', 'strasbourg', 'clinton', 'hindu',
        'defri', 'nep', 'derrida', 'politburo', 'francesca', 'parvi',
        'aldington', 'gassendi', 'exclaim', 'frankfurt', 'underwr

In [222]:
sort_s_c=uq_SB_words['s'][1][np.argsort(uq_SB_words['s'][1])[::-1]][:20000]

In [230]:
sort_s_c[~np.in1d(sort_s,sort_w)][:100]

array([1919,  568,  438,  306,  157,  146,  135,  118,  111,  103,   89,
         86,   86,   80,   73,   70,   67,   64,   63,   53,   52,   52,
         50,   49,   47,   45,   42,   42,   41,   39,   39,   39,   38,
         38,   38,   37,   37,   33,   33,   32,   32,   31,   31,   31,
         30,   29,   29,   28,   28,   28,   28,   27,   27,   26,   26,
         26,   25,   25,   25,   25,   25,   25,   24,   24,   24,   24,
         23,   23,   23,   22,   22,   22,   22,   22,   22,   22,   22,
         22,   21,   21,   21,   21,   21,   21,   21,   21,   21,   21,
         20,   20,   20,   20,   20,   20,   20,   20,   20,   19,   19,
         19])

In [263]:
np.where(sort_w == 'maintain'), np.where(sort_s == 'maintain')

((array([1287]),), (array([2807]),))

In [265]:
sort_w_c[1287], sort_s_c[22]

(477, 39301)

In [543]:
# word_string = ['maintain', 'keep', 'sort', 'put', 'inherit', 'get', 'obtain', 'up', 'down', 'shit',
#               'go', 'enhanc', 'enabl', 'nice', 'cute', 'good', 'can', 'stink', 'odour',
#               'sick', 'guy', 'bloke', 'see', 'rare', 'peril', 'vicious', 'wash', 'gum',
#               'tranquil', 'bold', 'brave', 'sort', 'set', 'feel','want', 'like', 'thus',
#               'wonder', 'do', 'make', 'cheap', 'busy', 'again', 'urg', 'rush', 'top',
#               'bottom', 'should', 'it', 'think', 'look', 'meat', 'insert', 'wide', 'ampl',
#               'mouth', 'fat', 'pleas', 'fart', 'blow', 'bastard', 'poor', 'sit', 'stay',
#               'super', 'atroc', 'pain', "catch", 'miss', 'mean', 'seek', 'sought', 'search',
#               'better', 'wors', 'neat', 'clean', 'skew', 'random', 'zero', 'bad', 'finish',
#               'end', 'commenc', 'trip', 'on', 'special', 'car', 'word', 'henc', 'well', 'by',
#               'peopl', 'give', 'back', 'some', 'most', 'just']

# print('word', 'spoken', 'written')
# for word in word_string:
#     print(word, sort_s_c[np.argwhere(sort_s == word)], sort_w_c[np.argwhere(sort_w == word)])

In [281]:
int_words=np.intersect1d(sort_s,sort_w)

In [544]:
l_w=[]
for word in int_words[:100]:
    i_w, i_s = np.argwhere(sort_w == word), np.argwhere(sort_s == word)
    count_w = sort_w_c[i_w][0][0]
    count_s = sort_s_c[i_s][0][0]
    if count_s and count_w:
        if (count_w >= 20 or count_s >= 20) & (count_w / count_s >=5):
            l_w.append((sort_w[i_w][0][0],
                        sort_s_c[i_s][0][0], 
                        sort_w_c[i_w][0][0]))

In [546]:
#l_w

In [386]:
l_s=[]
for word in int_words:
    i_w, i_s = np.argwhere(sort_w == word), np.argwhere(sort_s == word)
    count_w = sort_w_c[i_w][0][0]
    count_s = sort_s_c[i_s][0][0]
    if count_s and count_w:
        if (count_w >= 100 or count_s >= 100) & ( count_s / count_w >= 5):
            l_s.append((sort_s[i_s][0][0],
                        sort_s_c[i_s][0][0], 
                        sort_w_c[i_w][0][0]))

In [547]:
#l_s

In [700]:
a1 = np.array(['aa','rr','ww'])
a2 = np.array(['qq','aa','ii','ww','ll','hh','rr'])
a3= ['qq','aa','ii','ww','ll','hh','rr']
# wanted result [1, 6, 3]

In [701]:
%timeit [a3.index(x) for x in a1]

100000 loops, best of 3: 8.57 µs per loop


In [702]:
[np.where(a2==x)[0][0] for x in a1]

[1, 6, 3]

In [672]:
np.argwhere(a2 == a1[0])[0][0]

1

In [674]:
np.where(np.in1d(a2, a1))[0]

array([1, 3, 6])

In [775]:
#ANOTHER METHOD TO COMBINE S-W VOCAB

w_s = np.array(['aa','xx','cd','cc','ww','yy'])
w_s_counts = np.array([33,22,7,3,2,1], dtype=np.uint32)

w_w = np.array(['cc','bb','aa','cd','tw','ww'])
w_w_counts = np.array([25,18,14,9,5,1], dtype=np.uint32)
# find union
union = np.union1d(w_s, w_w)
union_c = np.zeros(union.shape[0], dtype=np.uint32)

In [776]:
t_s = [np.where(union==x)[0][0] for x in w_s]
t_w = [np.where(union==x)[0][0] for x in w_w]

union_c[t_s] += w_s_counts
union_c[t_w] += w_w_counts


In [779]:
ix_sort_union = np.argsort(union_c)[::-1]
union_sorted = union[ix_sort_union]
union_c_sorted = union_c[ix_sort_union]

In [780]:
t_s2 = [np.where(union_sorted==x)[0][0] for x in w_s]
t_w2 = [np.where(union_sorted==x)[0][0] for x in w_w]

In [782]:
# get samples fom each dist
r_s = np.random.randint(0,5, 6).astype(np.uint32)
r_w = np.random.randint(0,10, 6).astype(np.uint32)

union_c_sorted[t_s2] += r_s
union_c_sorted[t_w2] += r_w

union_sorted, union_c_sorted

### Merge and COMPRESS s and w vocabs: Application to REAL DATA

In [1019]:
# sort unique words and corresponding counts (limit s and w to 20k and 40k respectively)
sort_s = uq_SB_words['s'][0][np.argsort(uq_SB_words['s'][1])[::-1]][:20000]
sort_s_c=uq_SB_words['s'][1][np.argsort(uq_SB_words['s'][1])[::-1]][:20000].astype(np.uint32)

sort_w = uq_SB_words['w'][0][np.argsort(uq_SB_words['w'][1])[::-1]][:40000]
sort_w_c=uq_SB_words['w'][1][np.argsort(uq_SB_words['w'][1])[::-1]][:40000].astype(np.uint32)

In [2172]:
# plt.plot(sort_s_c)
# plt.xscale('log')
# plt.yscale('log')
plt.loglog(np.arange(len(sort_s_c)),sort_s_c)
plt.grid(True,which="both",ls="-")


In [2174]:
#compute intersection set length
np.intersect1d(sort_s, sort_w).shape

(15692,)

In [2187]:
plt.scatter(np.nonzero(np.in1d(sort_s, sort_w))[0], np.ones(15692))

In [2190]:
hs=np.histogram(np.nonzero(np.in1d(sort_s, sort_w))[0], bins=40)
plt.hist(np.nonzero(np.in1d(sort_s, sort_w))[0])

(array([ 1993.,  1970.,  1900.,  1808.,  1689.,  1546.,  1435.,  1258.,
         1140.,   953.]),
 array([     0. ,   1999.9,   3999.8,   5999.7,   7999.6,   9999.5,
         11999.4,  13999.3,  15999.2,  17999.1,  19999. ]),
 <a list of 10 Patch objects>)

In [821]:
# compute union of different word types and initialize total global counts
union_real = np.union1d(sort_s, sort_w)
union_real_c = np.zeros(union_real.shape[0], dtype=np.uint32)

In [1020]:
# compute transformations from lang_type sorted_lists to union_list
t_s = [np.where(union_real==x)[0][0] for x in sort_s]
t_w = [np.where(union_real==x)[0][0] for x in sort_w]

# update counts
union_real_c[t_s] += sort_s_c
union_real_c[t_w] += sort_w_c

In [1021]:
#sort union arrays
ix_sort_union = np.argsort(union_real_c)[::-1]
union_real_sorted = union_real[ix_sort_union]
union_real_c_sorted = union_real_c[ix_sort_union]

In [1488]:
#plot union vocabulary
plt.plot(union_real_c_sorted)
plt.xscale('log')
plt.yscale('log')

In [1022]:
# compute transformations from lang_type sorted_lists to SORTED union_list
t_s_un_sort = [np.where(union_real_sorted==x)[0][0] for x in sort_s]
t_w_un_sort = [np.where(union_real_sorted==x)[0][0] for x in sort_w]

In [856]:
t_s_un_sort = np.array(t_s_un_sort)
t_w_un_sort = np.array(t_w_un_sort)
transf_to_sorted_union = {'s':t_s_un_sort, 'w':t_w_un_sort}

In [857]:
dd.io.save('transf_to_sorted_vocab_union.h5', transf_to_sorted_union)

In [1023]:
transf_to_sorted_union = dd.io.load('transf_to_sorted_vocab_union.h5')

In [1036]:
# limit total words to 40000
t_s_un_sort_LIM = np.array([x for x in t_s_un_sort if x<40000])
t_w_un_sort_LIM = np.array([x for x in t_w_un_sort if x<40000])

In [2149]:
np.sort(np.union1d(t_s_un_sort_LIM, t_w_un_sort_LIM))

In [1038]:
(t_s_un_sort_LIM/40).astype(np.uint32), (t_w_un_sort_LIM/40).astype(np.uint32)

(array([  0,   0,   0, ..., 981, 567, 730], dtype=uint32),
 array([  0,   0,   0, ..., 993, 719, 993], dtype=uint32))

In [2150]:
plt.plot((t_s_un_sort_LIM/40).astype(np.uint32))

In [2151]:
df_s=pd.DataFrame({'s':(t_s_un_sort_LIM/40).astype(np.uint32)})
df_w=pd.DataFrame({'w':(t_w_un_sort_LIM/40).astype(np.uint32)})


In [2152]:
df_s.groupby('s').size().plot()

In [2153]:
df_w.groupby('w').size().plot()

In [945]:
randZipf(cdfs['s'][1000],10)

array([9, 8, 2, 0, 0, 0, 2, 0, 0, 0])

In [1254]:
ix_s=(np.array(t_s_un_sort_LIM)/40).astype(np.uint32) 
ix_w=(np.array(t_w_un_sort_LIM)/40).astype(np.uint32)

In [1255]:
ix_w.shape

(36487,)

In [8]:
os.chdir("/Users/PG/Paolo/python_repos/language_proj/lang_model_simple")
cdfs = dd.io.load('cdfs_3R_vs_step.h5')

In [990]:
i_s, c_s = np.unique(randZipf(cdfs['s'][1500], 1000), return_counts=True)
i_w, c_w = np.unique(randZipf(cdfs['w'][1500], 1000), return_counts=True)

In [1112]:
np.bincount(ix_w[0:40]),np.bincount(ix_w[40:80]),np.bincount(ix_w[80:120])

(array([30, 10]), array([ 6, 17, 13,  4]), array([ 1,  4, 16, 14,  3,  2]))

In [1301]:
36480/35, 19240/38

(1042.2857142857142, 506.3157894736842)

In [2154]:
#l_s_max_bins=np.array([(np.argmax(np.bincount(ix_s[i:i+40]))) for i in range(0,20000,40)])
#l_w_max_bins=np.array([(np.argmax(np.bincount(ix_w[i:i+40]))) for i in range(0,40000,40)])
l_s_max_bins = []
for i in range(0,19240,38):
    bin_c = np.bincount(ix_s[i:i+38])
    len_bins = np.nonzero(bin_c)[0].shape[0]
    idxs_maxs = heapq.nlargest(len_bins, range(len(bin_c)), bin_c.take)
    #print(idxs_maxs)
    for elem in idxs_maxs:
        if elem in l_s_max_bins:
            pass
        else:
            l_s_max_bins.append(elem)
            break
l_s_max_bins = np.array(l_s_max_bins)


In [2155]:
l_w_max_bins = []
for i in range(0,36480,35):
    bin_c = np.bincount(ix_w[i:i+35])
    len_bins = np.nonzero(bin_c)[0].shape[0]
    idxs_maxs = heapq.nlargest(len_bins, range(len(bin_c)), bin_c.take)
    for elem in idxs_maxs:
        if elem in l_w_max_bins:
            pass
        else:
            l_w_max_bins.append(elem)
            break
l_w_max_bins = np.array(l_w_max_bins)    


In [1355]:
len(l_s_max_bins), len(np.unique(l_s_max_bins)), len(l_w_max_bins), len(np.unique(l_w_max_bins))


(506, 506, 990, 990)

In [1356]:
plt.plot(l_s_max_bins)
plt.plot(l_w_max_bins)

In [1431]:
np.unique(np.union1d(l_s_max_bins, l_w_max_bins)).shape, np.unique(np.intersect1d(l_s_max_bins, l_w_max_bins)).shape

((997,), (499,))

In [1476]:
plt.plot(l_s_max_bins)

In [1050]:
l_w_max_bins.sort()
plt.plot(l_w_max_bins)

In [1059]:
ix_s[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 2, 2, 1, 2, 0, 1,
       2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 3, 1, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 3, 1], dtype=uint32)

In [982]:
np.bincount(ix_s[19240:19280])

array([], dtype=int64)

In [1117]:
np.union1d(l_s_max_bins, l_w_max_bins).sum()

381556

In [1357]:
np.unique(l_s_max_bins).shape, np.unique(l_w_max_bins).shape

((506,), (990,))

In [1125]:
a = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0])
ind = np.argpartition(a, -4)[-4:]
a[ind]


array([4, 9, 6, 9])

In [1875]:
plt.plot(cdfs['s'][1000])

In [2130]:
# get random samples from s, w
num_samp=5000000
r_z_s=randZipf(cdfs['s'][1000], num_samp)
r_z_w=randZipf(cdfs['w'][1000], num_samp)

In [2131]:
r_z_s_uq=np.unique(r_z_s, return_counts=True)
r_z_w_uq=np.unique(r_z_w, return_counts=True)

In [2137]:
plt.plot(r_z_w_uq[1])
plt.xscale('log')
plt.yscale('log')

In [2143]:
# initial method
merged_vocab=np.zeros(1000,dtype=np.int64)
merged_vocab[:r_z_s_uq[1].shape[0]] += r_z_s_uq[1]
merged_vocab[:r_z_w_uq[1].shape[0]] += r_z_w_uq[1]

In [1509]:
r_z_s_uq[1].shape, r_z_s_uq[1].shape[0]

((498,), 498)

In [2156]:
# mixed method 
merged_vocab=np.zeros(1000,dtype=np.int64)
merged_vocab[l_s_max_bins[r_z_s_uq[0]]] += r_z_s_uq[1]
merged_vocab[l_w_max_bins[r_z_w_uq[0]]] += r_z_w_uq[1]

In [2157]:
plt.plot(merged_vocab)

plt.xscale('log')
plt.yscale('log')

In [1491]:
#l_s_max_bins[r_z_s_uq[0]], l_w_max_bins[r_z_w_uq[0]]

In [ ]:
heapq.nlargest(len(r_z), range(len()))

In [1489]:
#np.unique(r_z, return_counts=True)

In [1379]:
z = np.zeros(500).astype(np.int64)


In [1490]:
#np.bincount(r_z)

In [1385]:
bc=np.bincount(r_z)

In [1428]:
%timeit z[:bc.shape[0]] += bc

The slowest run took 10.00 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 3.35 µs per loop


###### transform from s to w ( assuming w includes all s)

In [2239]:
rand_s = randZipf(cdfs['s'][1000],1000)

In [2253]:
np.bincount(rand_s) # much faster than np.unique ( factor ~ 16)

In [2251]:
np.nonzero(np.bincount(rand_s)) # np.nonzero + np.bincount ~~ 1/8 than np.unique(counts=True)

100000 loops, best of 3: 5.02 µs per loop


In [2245]:
np.unique(rand_s, return_counts=True) # slower than bincount

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  31,  32,  34,  35,  39,  40,  42,  44,  45,  46,
         47,  49,  52,  55,  56,  57,  58,  59,  62,  66,  67,  68,  72,
         75,  78,  80,  81,  83,  86,  89,  92,  93,  95,  97, 103, 114,
        118, 130, 132, 135, 137, 139, 141, 158, 167, 181, 201, 209, 214,
        230, 243, 254, 276, 300, 345, 356, 434]),
 array([494, 137,  64,  42,  35,  19,  18,  11,  12,  14,   8,  11,   7,
          4,   3,   4,   6,   5,   2,   7,   3,   2,   3,   3,   3,   3,
          2,   1,   2,   5,   2,   3,   2,   2,   2,   3,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   2,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   2,   1,   1,   1,   2,   1,   1,   1,   1,
          2,   2,   1,   1,   2,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1]))

In [2234]:
np.nonzero(np.in1d(sort_s, sort_w))

(array([    0,     1,     2, ..., 19994, 19997, 19999]),)

In [2193]:
t_s_w = [np.where(sort_w==x)[0][0] for x in sort_s if x in sort_w]

In [2198]:
plt.plot(pd.rolling_mean(np.array(t_s_w), 40))

In [2208]:
np.array(t_s_w)[:40]

array([  13,    0,   27,    6,   10,    3,    8,    2,    4,   45,   49,
       4143,    5,    1,   12,   26,    9,  667,   23,   55,   38,   63,
          7,   14,   37,  141,  116,   28,  167,   22,   71,   42,   11,
         16,   57,  271,   95,   50,   30,   24])

In [2211]:
sort_s[:40], sort_w[:40]

(array(['i', 'the', 'you', 'it', "'s", 'and', 'that', 'to', 'a', "n't",
        'do', 'yeah', 'in', 'of', 'he', 'they', 'is', 'oh', 'have', 'what',
        'we', 'no', 'was', 'on', 'there', 'well', 'know', 'she', 'got',
        'but', 'go', 'one', 'for', 'be', 'so', 've', 'get', 'like', 'this',
        'not'], 
       dtype='<U104'),
 array(['the', 'of', 'to', 'and', 'a', 'in', 'it', 'was', 'that', 'is',
        "'s", 'for', 'he', 'i', 'on', 'with', 'be', 'his', 'as', 'had',
        'at', 'by', 'but', 'have', 'not', 'from', 'they', 'you', 'she',
        'are', 'this', 'an', 'which', 'her', 'were', 'said', 'has', 'there',
        'we', 'would'], 
       dtype='<U25'))

In [2260]:
df = pd.DataFrame({"t_s_w": np.array(t_s_w)})

# Bin the data frame by "a" with 10 bins...
bins = np.linspace(df.t_s_w.min(), df.t_s_w.max(), 392)
groups = df.groupby(np.digitize(df.t_s_w, bins))

# Get the mean of each bin:
print (groups.mean()/40) # Also could do "groups.aggregate(np.mean)"

In [2232]:
(groups.mean()/40).round().t_s_w.values

In [2227]:
1000/392, 54000/24000, 15680/40

(2.5510204081632653, 2.25, 392.0)

### Assembling ALGO for WORD MEMORY

1. Vocab size changes with age, but power law expression scales to adapt to increasing vocab


In [9]:
os.chdir('/Users/PG/Paolo/python_repos/language_proj')

In [14]:
# THREE REGIMES FIT (SPOKEN)
sigma =np.ones(len(y_s))
sigma[[99, 100, 101, 500, 1700, 1999, 2000, 2001, 2002, -1]] = 0.001
(a3S_1, a3S_2,a3S_3,beta_3S, c3S_1, c3S_2, c3S_3, C4_S), _ = opt.curve_fit(zipf_mand_fit_3S, 
                                                                     x_s, y_s, 
                                                                     p0=(1., 1.3, 2.5, 
                                                                     3, 
                                                                     100000, 1000, 100, 10),
                                                                     sigma=sigma)
(a3S_1, a3S_2,a3S_3,beta_3S, c3S_1, c3S_2, c3S_3, C4_S)

(1.2222786840696638,
 1.4954601266269689,
 1.8986458118523217,
 7.7972793355069152,
 2613014.9799408969,
 9668.5668750285349,
 122.1289147117548,
 11.429085154568304)

In [15]:
# 3S WRITTEN FIT
sigma =np.ones(len(y_w))
sigma[[99, 100, 101, 500, 1700, 1999, 2000, 2001, 2002, -1]] = 0.001
(a3S_w1, a3S_w2,a3S_w3,beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w), _ = opt.curve_fit(zipf_mand_fit_3S, 
                                                            x_w, y_w, 
                                                            p0=(1.,1.3,2.5, 
                                                                3, 100000, 1000,100, 10),
                                                            sigma=sigma)
(a3S_w1, a3S_w2,a3S_w3,beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w)

(0.98779924555558052,
 0.78013700474286507,
 1.6496670400187725,
 0.60344865923415125,
 501780.35693374212,
 5544.6062437143801,
 279.60203407490104,
 -259.15257421842836)

In [16]:
# Define coefficients for power laws
# alpha exponents are no longer a function of age
# They are constant and it is vocab size that changes with age


# use values obtained with optimization
fit_coeffs = {'s':[a3S_1, a3S_2, a3S_3, beta_3S, c3S_1, c3S_2, c3S_3, C4_S], 
              'w':[a3S_w1, a3S_w2, a3S_w3, beta_3w, c3S_w1, c3S_w2, c3S_w3, C4_w]}

In [17]:
# find parameters of sigmoid for spoken vocab
from scipy.optimize import fsolve

s_final_size = 20000
def equations_s(p):
    p1, p2, p3 = p
    eqs = ((s_final_size + p3)/(1 + np.exp( -p1 * (0 - p2))) - p3 -1,
           (s_final_size + p3)/(1 + np.exp( -p1 * (150 - p2))) - p3 -5000,
           (s_final_size + p3)/(1 + np.exp( -p1 * (500 - p2))) - p3 -17000)
    return eqs

s1,s2,s3 =  fsolve(equations_s, (0.02, 300, 2000))

print (s1,s2,s3)

0.00694140842913 216.651070505 4444.17263094


In [2100]:
# find parameters of sigmoid for written vocab
w_final_size = 40000
def equations_w(p):
    p1, p2, p3 = p
    eqs = ((w_final_size + p3)/(1 + np.exp( -p1 * (0 - p2))) - p3 -2,
           (w_final_size + p3)/(1 + np.exp( -p1 * (150 - p2))) - p3 -10000,
           (w_final_size + p3)/(1 + np.exp( -p1 * (500 - p2))) - p3 -30000)
    return eqs

w1,w2,w3 =  fsolve(equations_w, (0.02, 300, 5000))

print (w1,w2,w3)

0.00458704924706 147.829452321 20300.1841713


In [2261]:
# Define age and max vocabulary sizes for both types
age = np.linspace(0,3600-1,3600)
max_vocab_size = {'s':20000, 'w':40000}

# use sigmoid to model vocab size evolution
#p1, p2 = 0.02, 260
p1, p2,  p3 = s1, s2, s3
s_vocab_size_vs_age = (max_vocab_size['s']+p3)/(1 + np.exp( -p1 * (age - p2))) - p3


p4, p5, p6 = w1,w2,w3
w_vocab_size_vs_age = (max_vocab_size['w'] + p6) /(1 + np.exp( -p4 * (age - p5))) -p6

#vocab_size vs age
vocab_size = {'s':s_vocab_size_vs_age.astype(np.uint32), 
              'w':w_vocab_size_vs_age.astype(np.uint32)}

def plot_vocab_size_vs_age():
    fu,fl = 1.1, 0.9
    fig, ax = plt.subplots(2,1)
    for i, key in zip([0,1],['s', 'w']):
        ax[i].plot(age, fu * vocab_size[key], label = str(round(100*fu,2))+'%')
        ax[i].plot(age, vocab_size[key])
        ax[i].plot(age, fl * vocab_size[key], label = str(round(100*fl,2))+'%')
        ax[i].set_xlabel('age')
        ax[i].set_ylabel('vocab_size_in_words')
        ax[i].legend(loc='lower right')
    
    ax[0].set_title('evolution of vocabulary size with age', size=15)
    
plot_vocab_size_vs_age()

Compute COMPRESSED CDFs for both 's' and 'w' as a function of age
    - Assume max size of vocabs is fixed
    - Apply ratio 20000 -> 500 = 40 to all sizes through age values

In [2102]:
cdfs = defaultdict(list)

for lang_type in ['s', 'w']:
    for num_words in vocab_size[lang_type]:
        cdfs[lang_type].append(Zipf_Mand_3S_CDF_comp(num_words, 
                                                     *fit_coeffs[lang_type], 
                                                     n_red=max(int(num_words/40) + 1,1)))

In [31]:
#Now compute FULL CDFS ( not compressed ) for comparison
cdfs_FV = defaultdict(list)

for lang_type in ['s', 'w']:
    for num_words in vocab_size[lang_type]:
        cdfs_FV[lang_type].append(Zipf_Mand_3S_CDF(num_words, 
                                                     *fit_coeffs[lang_type])

In [2262]:
plt.plot(cdfs['s'][1500],label='s')
plt.plot(cdfs['w'][2000],label='w')
plt.legend()



###### SAVE CDFS

In [2104]:
# Save compressed vocabulary CDFs ( speech and written)
# Need to pickle data before saving to hdf5 because deep dish cannot handle object array properly
# cdfs is a list of arrays of different shape !!!!!!!!
os.chdir('/Users/PG/Paolo/python_repos/language_proj/lang_model_simple/')

dd.io.save('cdfs_3R_vs_step.h5', dd.io.ForcePickle(cdfs))


In [18]:
os.chdir("/Users/PG/Paolo/python_repos/language_proj/lang_model_simple")
cdfs = dd.io.load('cdfs_3R_vs_step.h5')

DEFINE KEY FUNCTION FOR WORD MEMORY

In [2267]:
yy=[words_day_factor(age) for age in range(3000)]

In [2268]:
plt.plot(yy)

In [13]:
#MODIF

def words_day_factor(age):
    """ Define coeff that determines num hours spoken per day
        as pct of vocabulary size, assuming 16000 tokens per adult per day
        as average """
    if age < 36 * 14:
        return 2.5 + 100 * np.exp(-0.014 * age)
    elif 36 * 14 <= age <= 36 * 65:
        return 2.5
    elif age > 36 * 65:
        return 1.5 + np.exp(0.002 * (age - 36 * 65 ) )

# KEY MEMORY FUNCTION 
# LISTENING MODEL

def get_lang_knowledge(t, S, cdf_data, word_counter, num_steps1, num_steps2, 
                       pct_hours, lang_knowledge,
                       vocab_size=40000, a=7.6, b=0.023, c=-0.031, d=-0.2, n_red=1000,
                       pct_threshold=0.9, read_hours_per_day=0.5):
    """ Function to compute and update main arrays that define agent linguistic knowledge
    
        Args:
            * t: numpy array(shape=vocab_size) that counts elapsed steps from last activation of each word
            * S: numpy array(shape=vocab_size) that measures memory stability for each word
            * cdf_data: dict with keys 's','w'. It gives a CDF for each step
            * num_steps1, num_steps2: scalars. Initial and final steps for calculation
            * pct_hours: percentage of daily hours devoted to chosen language
            * lang_knowledge: array with shape = steps. For each step , 
              it gives percentage knowledge of language as measured 
              by a given threshold of retrievability(pct_threshold)
            * a, b, c, d: parameters to definE memory functioN from SUPERMEMO by Piotr A. Wozniak
        
        MEMORY MODEL: https://www.supermemo.com/articles/stability.htm
        
        Assumptions ( see "HOW MANY WORDS DO WE KNOW ???" By Marc Brysbaert*, 
        Michaël Stevens, Paweł Mandera and Emmanuel Keuleers): 
            * ~16000 spoken tokens per day + 16000 heard tokens per day + TV, RADIO
            * 1min reading -> 220-300 tokens with large individual differences, thus
              in 1 h we get ~ 16000 words"""
    
    #initialize R before looping if needed
    if not num_steps1:
        R = np.zeros(n_red)
    else:
        R = np.exp(-k * t/S)
    
    for age in range(num_steps1, num_steps2):

        # compute current average COMPRESSED number of words per day
        words_per_day = n_red / words_day_factor(age) # pack equation already in function ??
        # define correction factor for speech vocab to account for TV, own spoken words, etc...
        f_s = 2
        if age > 7 * 36 and random.random() > 0.1:
            zipf_samples = randZipf(cdf_data['s'][age], int(f_s * pct_hours * words_per_day * 10))
            zipf_samples_written = randZipf(cdf_data['w'][age], int(read_hours_per_day * pct_hours * (0.9*words_per_day) * 10))
            zipf_samples = np.concatenate((zipf_samples, zipf_samples_written))
        else:
            zipf_samples = randZipf(cdf_data['s'][age], int(f_s * pct_hours * words_per_day * 10))

        # assess which words and how many of each were encountered in current step
        act, act_c = np.unique(zipf_samples, return_counts=True)
        # update word counter according to previous line
        #np.add.at(word_counter, act, act_c)  # 
        word_counter[act] += act_c
        # check which words are available for memorization ( need minimum number of times)
        mem_availab_words = np.where(word_counter > 5)[0]
        
        # compute indices of active words that are available for memory 
        idxs_act = np.nonzero(np.in1d(act, mem_availab_words, assume_unique=True))
        # get really activated words
        act = act[idxs_act]
        # Apply indices to counts ( retrieve only counts of really active words)
        act_c = act_c[idxs_act] 


        if act.any():  # check that there are any real active words            
            # compute increase in memory stability S due to (re)activation
            delta_S = a * (S[act]**(-b)) * np.exp(c * 100 * R[act]) + d
            # update memory stability value
            #np.add.at(S, act, delta_S) # 
            S[act] += delta_S 
            
            #define mask to update elapsed-steps array t
            mask = np.zeros(n_red, dtype=np.bool)
            mask[act] = True
            t[~mask] += 1 # add ones to last activation time counter if word not act
            t[mask] = 0 # set last activation time counter to one if word act
            
            #discount one to counts
            act_c -= 1
            #Simplification with good approx : we apply delta_S without iteration !!
            delta_S = act_c * (a * (S[act]**(-b)) * np.exp(c * 100 * R[act]) + d)
            #np.add.at(S, act, delta_S) # 
            S[act] += delta_S 

        else:
            #define mask to update elapsed-steps array t
            mask = np.zeros(n_red, dtype=np.bool)
            mask[act] = True
            t[~mask] += 1 # add ones to last activation time counter if word not act
            t[mask] = 0 # set last activation time counter to one if word act           
            
        # memory becomes ever shakier after turning 65...
        if age > 65 * 36:
            S = np.where(S >= 0.01, S - 0.01, 0.000001)
        # compute memory retrievability R from t, S
        R = np.exp(-k * t/S)
        lang_knowledge[age] = np.where(R > pct_threshold)[0].shape[0] / n_red
        
    return t, S, lang_knowledge, word_counter

In [14]:
def study_memory_fun(lang_path, cdfs=cdfs, n_red=1000, k = np.log(10/9), rhpd=0.5):
    S = np.full(n_red, 0.01)
    t = np.full(n_red, 100, dtype=np.int64)
    lang_knowledge = np.zeros(3000)
    word_counter = np.zeros(n_red, dtype=np.int64)
    
    for pcts, steps in zip(lang_path['pcts'], zip(lang_path['steps'], lang_path['steps'][1:])):
        t, S, lang_knowledge, word_counter =  get_lang_knowledge(t, S, cdfs, word_counter, 
                                                   *steps, pcts, lang_knowledge, n_red=n_red,
                                                   pct_threshold=0.9, read_hours_per_day=rhpd)
        
    
    return t, S, lang_knowledge, word_counter

In [1833]:
k = np.log(10/9)
lps = [{'pcts':[1,0.9,0.1, 0.4],'steps':[0, 500, 800, 1500, 3000]},
       {'pcts':[0,0.1,0.9, 0.6],'steps':[0, 500, 800, 1500, 3000]}]

list_t = []
list_S = []
list_lang_evols = []

for lp in lps:
    t, S, lang_evol, wc = study_memory_fun(lp)
    list_t.append(t)
    list_S.append(S)
    list_lang_evols.append(lang_evol)

In [1834]:
plt.plot(list_lang_evols[0],label='pol')
plt.plot(list_lang_evols[1],label = 'eng')
plt.legend()

In [2316]:
lps2 = [{'pcts':[0.5],'steps':[0, 3000]},
        {'pcts':[1], 'steps':[0, 3000]}]

list_lang_evols2 = []
list_t2 = []
list_S2 = []
for lp in lps2:
    x = study_memory_fun(lp)
    list_t2.append(x[0])
    list_S2.append(x[1])
    list_lang_evols2.append(x[2])

In [2317]:
# plt.plot(list_lang_evols2[0],label= 'fr')
# plt.plot(list_lang_evols2[1],label = 'hung')
plt.plot(list_lang_evols2[0],label = '50%')
plt.plot(list_lang_evols2[1],label = 'cat')
plt.legend()

In [2315]:
#np.exp(-k * list_t2[3]/ list_S2[3])

In [2146]:
#plot rolling mean
plt.plot(pd.rolling_mean(list_lang_evols2[3], 20))

###### Generate Initial Conditions for ABM model

In [24]:
lang_paths_IC = [{'pcts':[0.25],'steps':[0, 1500]},
        {'pcts':[0.5],'steps':[0, 1500]},
        {'pcts':[0.75],'steps':[0, 1500]},
        {'pcts':[1], 'steps':[0, 1500]}]

t_IC = []
S_IC = []
lang_hist = []
wc_IC = []
for lp in lang_paths_IC:
    lang_res = study_memory_fun(lp, rhpd=0.)
    t_IC.append(lang_res[0])
    S_IC.append(lang_res[1])
    lang_hist.append(lang_res[2])
    wc_IC.append(lang_res[3])
t_IC = np.array(t_IC)
S_IC = np.array(S_IC)
lang_hist = np.array(lang_hist)
wc_IC = np.array(wc_IC)

In [27]:
plt.plot(lang_hist[0][:1500])
plt.plot(lang_hist[3][:1500])

In [18]:
IC_lang = {}

IC_lang['25_pct'] = {'t':t_IC[0],'S':S_IC[0], 'wc':wc_IC[0]}
IC_lang['50_pct'] = {'t':t_IC[1],'S':S_IC[1], 'wc':wc_IC[1]}
IC_lang['75_pct'] = {'t':t_IC[2],'S':S_IC[2], 'wc':wc_IC[2]}
IC_lang['100_pct'] = {'t':t_IC[3],'S':S_IC[3], 'wc':wc_IC[3]}

In [2395]:
# reading + speaking up to 41 years
lang_paths_IC = [{'pcts':[0.25],'steps':[0, 1500]},
        {'pcts':[0.5],'steps':[0, 1500]},
        {'pcts':[0.75],'steps':[0, 1500]},
        {'pcts':[1], 'steps':[0, 1500]}]



t_IC = []
S_IC = []
wc_IC = []
for lp in lang_paths_IC:
    t_IC.append(study_memory_fun(lp)[0])
    S_IC.append(study_memory_fun(lp)[1])
    wc_IC.append(study_memory_fun(lp)[3])
t_IC = np.array(t_IC)
S_IC = np.array(S_IC)
wc_IC = np.array(wc_IC)

In [2401]:
# IC_lang = {}

# IC_lang['25_pct'] = {'t':t_IC[0],'S':S_IC[0], 'wc':wc_IC[0]}
# IC_lang['50_pct'] = {'t':t_IC[1],'S':S_IC[1], 'wc':wc_IC[1]}
# IC_lang['75_pct'] = {'t':t_IC[2],'S':S_IC[2], 'wc':wc_IC[2]}
# IC_lang['100_pct'] = {'t':t_IC[3],'S':S_IC[3], 'wc':wc_IC[3]}
# os.chdir('/Users/PG/Paolo/python_repos/language_proj/lang_model_simple')
# dd.io.save('IC_lang_1500_steps.h5', IC_lang)

In [2397]:
IC_imp = dd.io.load('IC_lang_1500_steps.h5')

In [2400]:
IC_imp['25_pct']['wc'].dtype

dtype('int64')

###### COMPARISON REDUCED VOCABULARY (RV) VS FULL VOCABULARY(FV)

In [1447]:
list_lang_evols_RV = []
lps_RV = [{'pcts':[1, 0.25],'steps':[0, 500, 1000]}]
for lp in lps_RV:
    list_lang_evols_RV.append(study_memory_fun(lp)[2])

In [1440]:
list_lang_evols_FV = []
lps_FV = [{'pcts':[1, 0.25],'steps':[0, 500, 1000]}]
for lp in lps_FV :
    list_lang_evols_FV.append(study_memory_fun(lp, cdfs=cdfs_FV, n_red=40000)[2])

In [1441]:
plt.plot(list_lang_evols_RV[0][:1000],label='RV')
plt.plot(list_lang_evols_FV[0][:1000],label='FV')
plt.legend()

# conclusion : exactly same macro-trend, 
# RV shows more local oscillations due to reduced vocab size

Measuring performance

In [162]:
%load_ext line_profiler

In [741]:
S_1 = np.full(n_red, 0.01)
t_1 = np.full(n_red, 100, dtype=np.int64)
lang_knowledge_1 = np.zeros(3000)
word_counter_1 = np.zeros(n_red, dtype=np.int32)
%lprun -f get_lang_knowledge get_lang_knowledge(t_1, S_1, cdf_data_zpf_mand, word_counter_1, 0, 200, 1, lang_knowledge_1, vocab_size)

In [82]:
#plt.plot(lang_knowledge,label="z")
plt.plot(lang_knowledge,label="base")
# plt.plot(lang_knowledge_zm,label="zm")
# plt.plot(lang_knowledge_zm2,label="zm2")
# plt.plot(lang_knowledge_zm3,label="zm3")
# plt.plot(lang_knowledge_025,label="zm4")
# plt.xlabel('steps')
# plt.ylabel('% knowledge')
# plt.ylim(0,1)
# plt.legend()
# plt.grid('on')
# plt.show()

In [706]:
np.average(S_zm[1:100], weights=1/np.arange(1,100)), np.average(S_zm2[1:100], weights=1/np.arange(1,100))

(8785.9887487181149, 8939.8527583142732)

In [160]:
def speak_model():
    # SPEAKING MODELING
    t_imp = IC_imp['75_pct']['t']
    S_imp = IC_imp['75_pct']['S']
    R = np.exp( - k * t_imp/S_imp)
    f = 1000 / words_day_factor(1500)

    # sample must come from AVAILABLE listened words !!!! This can be done in two steps

    # 1. First sample from lang CDF ( that encapsulates all needed concepts)
    zipf_samples = randZipf(cdf_data_zpf_mand['speech'][1500], int(f * 10))
    act, act_c = np.unique(zipf_samples, return_counts=True)
    # 2. Then compare to known words
    mask_R = np.random.rand(R[act].shape[0]) <= R[l1][act] # get words index from available ones

    s_words = act[mask_R]
    
    #if shapes are different and BOTH agents are bilingual, try to retrieve word from other language
    
    if s_words.shape != act.shape:
        miss_words = R[~mask_R] 
        R[~mask_R] < R[l2][act]



In [2334]:
 np.random.rand(10) < np.random.rand(10)

array([ True, False, False, False, False, False, False, False, False, False], dtype=bool)

In [513]:
# SPEAK PROCESS

R2 = np.exp(-k * t_zm2 / S_zm2)

#1. Get zipf samples from CDF
zipf_cum_distr = cdf_data_zpf_mand['speech'][36*18]
numSamples = 50
np.unique(randZipf(zipf_cum_distr, numSamples), return_counts=True)

np.intersect1d(np.where(R2>0.9), [301,406,806])

np.argwhere(u_r_array > 0.9).reshape(51,)

u_r_array = np.random.random(R2.shape[0])
R2 > u_r_array

m = R2[np.where(R2>0.9)].shape[0]
u_r_array = np.random.random(m)
np.searchsorted(R2[np.where(R2>0.9)], u_r_array)

u_r_array < R2[np.where(R2>0.9)]

### TEXT ANALYSIS

In [1]:
import nltk
from nltk.tokenize import word_tokenize
import requests
from bs4 import BeautifulSoup
import re

In [251]:
cat_text = "— Bon día; quina calor que fa avuy! nos pot anar.— Míri, ara mateix una servidora n'estava parlant; no's pot resistir.— Oy, qu'es cert; vamos, estigan bones: me'n vaig a posar a la fresca.— Vagi, vagi, estíga bona, senyora Marieta. Y encara no han sentit tancar la porta del pis, ja pregunta una ab veu baxa:—¿Què'ls hi sembla?— Dòna, lo que deyam; ab tanta roba com porta a sobre, rès li fa goig; tan esllanguida...— Si ella es més prima qu'un fus.— Oy tal! sembla un paraygua plegat; tot li baldeja."

In [224]:
#word_tokenize(cat_text)

In [247]:
cat_text = re.sub('[^a-zA-Z]', '', cat_text)
#cat_text = re.sub('[\d]', '', cat_text) 

In [252]:
cat_text = re.sub("[^\w']", ' ', cat_text)
cat_text

"  Bon día  quina calor que fa avuy  nos pot anar   Míri  ara mateix una servidora n'estava parlant  no's pot resistir   Oy  qu'es cert  vamos  estigan bones  me'n vaig a posar a la fresca   Vagi  vagi  estíga bona  senyora Marieta  Y encara no han sentit tancar la porta del pis  ja pregunta una ab veu baxa   Què'ls hi sembla   Dòna  lo que deyam  ab tanta roba com porta a sobre  rès li fa goig  tan esllanguida     Si ella es més prima qu'un fus   Oy tal  sembla un paraygua plegat  tot li baldeja "

In [253]:
len(word_tokenize(cat_text))

92

In [254]:
np.unique(word_tokenize(cat_text)).shape[0]

79

In [255]:
cat_text2 = "L'endemà, — continua la matexa, — qu'era diumenge, axís que ella tornà de missa, surto al celobert y pregunto a l'Antonieta si havía vist los estucadors; com aquella també es un poch Tana, que sembla Santa Reparada, no'm va entendre fins que li vaig haver signat. ¿Ne volen de rialles?... y com per assò també es molt ditxosa,'m fa ab molt desdeny: «¡Ay, filla! no me digas rès que tinch l'home més cremat qu'una torradora.» «¿Y axò?» «Rès, per que com es tan escrupulós, y ja se sab que tot li fa fàstich, diu qu'avuy se'n anirà a la fonda axis que senti fregir sardina.» «¡Ay, ay! ¿que n'has comprada?» «Nó, filla, nó; però diu que pel vehinat ha vist que n'enfarinavan!...» Y no va poguer dir més per que'l seu home la va fer ficar dins; com es tan sorrut y de pocus amigus allò..., ò sinó, ja'ls dich que s'hi hagueran pogut llogar cadires."

In [256]:
cat_text2 = re.sub("[^\w']", ' ', cat_text2)
cat_text2

"L'endemà    continua la matexa    qu'era diumenge  axís que ella tornà de missa  surto al celobert y pregunto a l'Antonieta si havía vist los estucadors  com aquella també es un poch Tana  que sembla Santa Reparada  no'm va entendre fins que li vaig haver signat   Ne volen de rialles     y com per assò també es molt ditxosa 'm fa ab molt desdeny    Ay  filla  no me digas rès que tinch l'home més cremat qu'una torradora     Y axò    Rès  per que com es tan escrupulós  y ja se sab que tot li fa fàstich  diu qu'avuy se'n anirà a la fonda axis que senti fregir sardina     Ay  ay   que n'has comprada    Nó  filla  nó  però diu que pel vehinat ha vist que n'enfarinavan      Y no va poguer dir més per que'l seu home la va fer ficar dins  com es tan sorrut y de pocus amigus allò     ò sinó  ja'ls dich que s'hi hagueran pogut llogar cadires "

In [257]:
len(word_tokenize(cat_text2))

156

In [258]:
np.unique(word_tokenize(cat_text2)).shape[0]

114

### HEAPS LAW ( Type-Token relation)

In [140]:
import requests
import re
from bs4 import BeautifulSoup

In [286]:
URL = "https://ca.wikisource.org/wiki/Del_meu_tros_-_La_viuda"
URL2 = "https://ca.wikisource.org/wiki/Del_meu_tros_-_No_n%27hi_hà_d%27altre"
URL3 = "https://en.wikipedia.org/wiki/1980_Formula_One_season"
html = requests.get(URL).text
soup = BeautifulSoup(html, 'xml')

In [287]:
def get_text_from_url(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'xml')
    text = [x.text for x in soup.body.find_all('p')]
    text = ''.join(text)
    text = re.sub("[^\w']", ' ', text)    
    return text

In [362]:
import pandas as pd
from bs4 import BeautifulSoup
class url_data():
    
    def get_soup(self, url):
        html = requests.get(url).text
        self.soup = BeautifulSoup(html, 'xml')
        
    def get_clean_text(self):
        self.text = [x.text for x in self.soup.body.find_all('p')]
        self.text = ''.join(self.text)
        self.text = re.sub("[^\w']", ' ', self.text)
        
    def get_tables(self, i):
        self.table = self.soup.body.find_all('table')[i]
        self.table = pd.read_html(str(self.table.find_all('table')))[0]
        
        

In [364]:
url_d1 = url_data()
url_d1.get_soup(URL3)
url_d1.get_tables(3)

In [378]:
url_d1.soup.find_all('table')[3].table.span.a

<a href="/wiki/Argentina" title="Argentina"><img alt="Argentina" class="thumbborder" data-file-height="500" data-file-width="800" height="14" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_Argentina.svg/23px-Flag_of_Argentina.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_Argentina.svg/35px-Flag_of_Argentina.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_Argentina.svg/46px-Flag_of_Argentina.svg.png 2x" width="23"/></a>

In [330]:
cat_text4 = get_text_from_url(URL2)

In [266]:
cat_text3 = [x.text for x in soup.body.find_all('p')]
cat_text3 = ''.join(cat_text3)

In [323]:
cat_text3 = re.sub("[^\w']", ' ', cat_text3)

In [324]:
n=6500
np.unique(word_tokenize(cat_text3)[:n]).shape[0]/n

0.2803076923076923

In [307]:
text_tokens = word_tokenize(cat_text3)
heaps_empiric = [(n, np.unique(text_tokens[:n]).shape[0], np.unique(text_tokens[:n]).shape[0]/n)
                 for n in range(50, len(text_tokens))]

In [338]:
def plot_text_heaps(text):
    text_tokens = word_tokenize(text)
    heaps_empiric = [(n, np.unique(text_tokens[:n]).shape[0], np.unique(text_tokens[:n]).shape[0]/n)
                 for n in range(50, len(text_tokens))]
    x, y1, y2 =list(zip(*heaps_empiric))
    (k,beta), _ = opt.curve_fit(heaps_fun, x, y1, p0=(10,0.5))
    ax1 = plt.subplot(211)
    ax1.plot(x,y1)
    ax1.set_xlabel('num_words')
    ax1.set_ylabel('num_types')
    ax2 = plt.subplot(212, sharex=ax1 )
    ax2.plot(x, y2)
    plt.show()
    return k, beta
    

In [438]:
plot_text_heaps(cat_text3)

(1.960793571386521, 0.79519369571849796)

In [308]:
x, y1, y2 =list(zip(*heaps_empiric))

In [309]:
import matplotlib.pyplot as plt
%matplotlib
ax1 = plt.subplot(211)
ax1.plot(x,y1)
ax1.set_xlabel('num_words')
ax1.set_ylabel('num_types')
ax2 = plt.subplot(212)
ax2.plot(x, y2)

Using matplotlib backend: MacOSX


In [288]:
np.unique(word_tokenize(cat_text3)[:6500]).shape[0]

1869

In [314]:
sp_words = np.random.normal(16000,7500,size=100)
plt.plot(sp_words)

In [315]:
import scipy.optimize as opt

def heaps_fun(x, k, beta):
    x = np.array(x)
    return k * x ** beta

In [319]:
(k,beta), _ = opt.curve_fit(heaps_fun, x, y, p0=(10,0.5))

In [321]:
k, beta

(1.7960287211231736, 0.7878744493804708)

In [322]:
plt.plot(x,k*np.array(x)**beta)
plt.plot(x,y)

In [344]:
x = np.linspace(1,1000,1000)
y = 1/(x + 2.7) ** 1.3
plt.plot(x, y)
plt.xscale('log')
plt.yscale('log')

In [1080]:
plt.plot(Zipf_Mandelbrot_3S_CDF(40000,fit_coeffs['w'][0],fit_coeffs['w'][1],fit_coeffs['w'][2]))

In [741]:
for num_w in [100,1000,5000,15000,24000]:
    plt.plot(Zipf_Mandelbrot_3S_CDF(num_w),label=str(num_w))
plt.legend()


In [539]:
n=20000
plt.plot(Zipf_Mandelbrot_CDF(n,1.32,4.46),label='old')
plt.plot(Zipf_Mandelbrot_CDF(n,1.3,8.78),label='ZM')
plt.plot(Zipf_Mandelbrot_3S_CDF(n),label='ZM_3S')
plt.legend()

In [575]:
np.unique(randZipf(Zipf_Mandelbrot_3S_CDF(n),16000*50),return_counts=True)[1].shape

(16136,)

In [574]:
np.unique(randZipf(Zipf_Mandelbrot_CDF(n,1.32,4.46),16000*50),return_counts=True)[1].shape

(17362,)

In [209]:
#BOOKS
n_b = 120000
alpha_b = 1.05
z_cdf = Zipf_CDF(n_b, alpha_b)
num_words_sample = 1002504
np.unique(randZipf(z_cdf, num_words_sample)).shape[0] / num_words_sample

0.08138221892381477

In [203]:
#SPEECH
n_s = 14700
alpha_s = 1.25
z_cdf = Zipf_CDF(n_s, alpha_s)
num_words_sample = 124800
np.unique(randZipf(z_cdf, num_words_sample)).shape[0] / num_words_sample

0.05479967948717949

In [432]:
n=30000
alpha=1.05
zpf_man_CDF=Zipf_Mandelbrot_CDF(n, alpha)
zpf_CDF = Zipf_CDF(n, alpha)
zpf_compressed = Zipf_CDF_compressed(n, alpha, n_red=1000)
beta = 2.7
zpf_mand_compressed = Zipf_Mand_CDF_compressed(n, alpha, beta, n_red=1000)

In [433]:
num_real_words =300
num_comp_words = num_real_words/30

In [434]:
np.mean([np.unique(randZipf(zpf_man_CDF, num_real_words)).shape 
         for _ in range(1000)]) / num_real_words

0.73437999999999992

In [435]:
np.mean([np.unique(randZipf(zpf_CDF, num_real_words)).shape 
         for _ in range(1000)]) / num_real_words

0.61629333333333336

In [436]:
np.mean([np.unique(randZipf(zpf_compressed, num_comp_words)).shape 
         for _ in range(1000)]) / num_comp_words

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:51: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.63300000000000001

In [437]:
np.mean([np.unique(randZipf(zpf_mand_compressed, num_comp_words)).shape 
         for _ in range(1000)]) / num_comp_words

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:51: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.76019999999999999

### RUN MODEL

In [2336]:
import numpy as np
import pandas as pd
import matplotlib
#matplotlib.use("TKAgg")
from matplotlib import pyplot as plt
%matplotlib
from matplotlib import animation

import os
os.chdir('/Users/PG/Paolo/python_repos/language_proj/lang_model_simple')

Using matplotlib backend: MacOSX


In [2363]:
from imp import reload
import agent_simple
import model_simple
reload(agent_simple)
reload(model_simple)
from model_simple import Simple_Language_Model
m1 = Simple_Language_Model(100,width=25, height=25, num_cities=3,
                           init_lang_distrib=[0.3, 0.5, 0.2],
                           lang_ags_sorted_by_dist=False, lang_ags_sorted_in_clust=False)
m1.run_model(100, recording_steps_period=20)

0%                          100%
[#                             ] | ETA: 00:00:26//anaconda/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:03


In [2350]:
yy, uu= None, None

In [2349]:
if not yy:
    print('ll')

ll
